# 📊 Data Exploration: NBA APIs

### 🗓 Date: 2025-07-16
### 🧑 Author: Sean Hingco

---

## 🎯 Objective
- Explore available NBA data APIs and their structure
- Evaluate the quality, granularity, and consistency of the data
- Identify what data fields are useful for Transformer model
- Draft an initial **data schema** for production use

---

## 🔍 APIs to Test
- [ ] ESPN API
- [ ] SportsRadar (if accessible)
- [ ] Other open datasets (e.g., Kaggle)

---

## 📝 Questions to Answer
- What player-level stats are available? (per game, per season, advanced metrics?)
- What game-level context can I get? (home/away, playoff, injuries, lineups?)
- How real-time or delayed is the data?
- How consistent are field names & formats between sources?

---

## 📐 Desired Data Schema (Draft)
- Game metadata:
  - `game_id`, `date`, `home_team`, `away_team`, `venue`, `season`, `playoff_flag`
- Team stats:
  - `team_id`, `points`, `rebounds`, `assists`, `etc.`
- Player stats:
  - `player_id`, `team_id`, `minutes`, `points`, `rebounds`, `FG%`, `advanced metrics`
- Lineups / rotations:
  - `TBD`

---

## 🧪 Notes & Observations
*ESPN Scoreboard Endpoint: https://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard*
* *provides game data and date parameter*
* *does not provide full roster data, just team and game leaders*

*ESPN Game Summary Endpoint: https://site.api.espn.com/apis/site/v2/sports/basketball/nba/summary?event={game_id}*
* *provides player statlines and advanced metrics can be derived from these*
* *no advanced metrics to provide information about defensive impact (another data source required)*

*NBA API: https://github.com/swar/nba_api*
* *provides player statlines and advanced metrics*
* *more robust metrics than ESPN API*
* *concerns with load management and rate limiting*
* *four endpoints to collect all box score data*

---

---

## ESPN Hidden API: Initial Exploration

- Endpoint: https://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard
- Goal: Understand data structure and evaluate if it meets our needs.
- Parameters: `dates=YYYYMMDD`

In [2]:
import requests
import json
from pprint import pprint

In [6]:
# initialize parameters with SCOREBOARD ENDPOINT
url = 'https://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard'

params = {
    "dates": "20250124"
}

In [7]:
# retrieve response and test
response = requests.get(url, params=params)
response.status_code

200

In [11]:
# parse, save, and view json
data = response.json()
pprint(data)

with open("espn_scoreboard_20250124.json", "w") as f:
    json.dump(data, f, indent=2)

{'events': [{'competitions': [{'attendance': 16228,
                               'broadcast': '',
                               'broadcasts': [{'market': 'away',
                                               'names': ['NBA League Pass']},
                                              {'market': 'home',
                                               'names': ['FanDuel SN SE',
                                                         'KUNP 16',
                                                         'KATU 2.2',
                                                         'BlazerVision']}],
                               'competitors': [{'homeAway': 'home',
                                                'id': '30',
                                                'leaders': [{'abbreviation': 'Pts',
                                                             'displayName': 'Points',
                                                             'leaders': [{'athlete': {'active': True,
     

In [13]:
# explore data structure
print(data.keys())
print(len(data['events']))

dict_keys(['leagues', 'events'])
3


In [28]:
data['events'][0]['competitions'][0]

{'id': '401705196',
 'uid': 's:40~l:46~e:401705196~c:401705196',
 'date': '2025-01-25T00:00Z',
 'attendance': 16228,
 'type': {'id': '1', 'abbreviation': 'STD'},
 'timeValid': True,
 'neutralSite': False,
 'conferenceCompetition': False,
 'playByPlayAvailable': True,
 'recent': False,
 'venue': {'id': '1893',
  'fullName': 'Spectrum Center',
  'address': {'city': 'Charlotte', 'state': 'NC'},
  'indoor': True},
 'competitors': [{'id': '30',
   'uid': 's:40~l:46~t:30',
   'type': 'team',
   'order': 0,
   'homeAway': 'home',
   'winner': False,
   'team': {'id': '30',
    'uid': 's:40~l:46~t:30',
    'location': 'Charlotte',
    'name': 'Hornets',
    'abbreviation': 'CHA',
    'displayName': 'Charlotte Hornets',
    'shortDisplayName': 'Hornets',
    'color': '008ca8',
    'alternateColor': '1d1060',
    'isActive': True,
    'venue': {'id': '1893'},
    'links': [{'rel': ['clubhouse', 'desktop', 'team'],
      'href': 'https://www.espn.com/nba/team/_/name/cha/charlotte-hornets',
      

In [26]:
def list_games(data):
    for e in data['events']:
        print(f"event: {e['name']}\ndate: {e['date']}\nstatus: {e['status']['type']['description']}")
        print()

In [27]:
list_games(data)

event: Portland Trail Blazers at Charlotte Hornets
date: 2025-01-25T00:00Z
status: Final

event: Cleveland Cavaliers at Philadelphia 76ers
date: 2025-01-25T00:00Z
status: Final

event: New Orleans Pelicans at Memphis Grizzlies
date: 2025-01-25T01:00Z
status: Final



In [32]:
# observe individual game data
game_id = data['events'][0]['competitions'][0]['id']
summary_url = f'https://site.api.espn.com/apis/site/v2/sports/basketball/nba/summary?event={game_id}'


In [33]:
response = requests.get(summary_url)
if response.status_code == 200:
    summary = response.json()
else:
    raise Exception(f"Failed to fetch summary: {response.status_code}")

In [35]:
pprint(summary.keys())
pprint(summary['boxscore'].keys())

dict_keys(['boxscore', 'format', 'gameInfo', 'leaders', 'seasonseries', 'injuries', 'broadcasts', 'pickcenter', 'againstTheSpread', 'odds', 'news', 'header', 'article', 'videos', 'winprobability', 'plays', 'wallclockAvailable', 'meta', 'standings'])
dict_keys(['teams', 'players'])


In [47]:
def save_summary_to_json(summary, game_id):
    filename = f"game_summary_{game_id}.json"
    with open(filename, "w") as f:
        json.dump(summary, f, indent=2)
    print(f"Saved to {filename}")

In [48]:
save_summary_to_json(summary, 401705196), 

Saved to game_summary_401705196.json


(None,)

In [36]:
# boxscore key has list of players
pprint(summary['boxscore']['players'])

[{'displayOrder': 1,
  'statistics': [{'athletes': [{'active': True,
                                'athlete': {'displayName': 'Toumani Camara',
                                            'guid': '29bc5947-de2e-3a0a-abe0-05de2767e34a',
                                            'headshot': {'alt': 'Toumani '
                                                                'Camara',
                                                         'href': 'https://a.espncdn.com/i/headshots/nba/players/full/4431736.png'},
                                            'id': '4431736',
                                            'jersey': '33',
                                            'links': [{'href': 'https://www.espn.com/nba/player/_/id/4431736/toumani-camara',
                                                       'rel': ['playercard',
                                                               'desktop',
                                                               'athlete'],
        

In [53]:
# to get player data for a game
def get_player_data(summary):
    for team in summary['boxscore']['players']:
        team_name = team['team']['displayName']
        print(f"\n=== {team_name} ===")
        stat_labels = team['statistics'][0]['labels']
        for player in team['statistics'][0]['athletes']:
            dnp_bool = player['didNotPlay']
            dnp = 'DNP' if dnp_bool else 'Played'
            reason = player['reason']
            position = player['athlete']['position']['abbreviation']
            name = player['athlete']['displayName']
            stats = player['stats']
            print(f"{name} — {position}: {[(sl, s) for sl, s in zip(stat_labels, stats)]}")
            print(f"{dnp} bc {reason}")

In [54]:
get_player_data(summary)


=== Portland Trail Blazers ===
Toumani Camara — F: [('MIN', '34'), ('FG', '5-10'), ('3PT', '1-4'), ('FT', '0-1'), ('OREB', '3'), ('DREB', '3'), ('REB', '6'), ('AST', '1'), ('STL', '2'), ('BLK', '0'), ('TO', '0'), ('PF', '4'), ('+/-', '+2'), ('PTS', '11')]
Played bc COACH'S DECISION
Jerami Grant — SF: [('MIN', '34'), ('FG', '7-18'), ('3PT', '4-8'), ('FT', '4-4'), ('OREB', '3'), ('DREB', '1'), ('REB', '4'), ('AST', '1'), ('STL', '1'), ('BLK', '0'), ('TO', '2'), ('PF', '1'), ('+/-', '+22'), ('PTS', '22')]
Played bc COACH'S DECISION
Deni Avdija — SF: [('MIN', '34'), ('FG', '3-6'), ('3PT', '1-3'), ('FT', '11-14'), ('OREB', '0'), ('DREB', '5'), ('REB', '5'), ('AST', '4'), ('STL', '4'), ('BLK', '0'), ('TO', '5'), ('PF', '2'), ('+/-', '+2'), ('PTS', '18')]
Played bc COACH'S DECISION
Donovan Clingan — C: [('MIN', '18'), ('FG', '2-4'), ('3PT', '0-1'), ('FT', '0-0'), ('OREB', '4'), ('DREB', '9'), ('REB', '13'), ('AST', '4'), ('STL', '1'), ('BLK', '4'), ('TO', '2'), ('PF', '1'), ('+/-', '+10'), (

## NBA API: Initial Exploration

- Endpoint: swar/nba_api
- Goal: Understand data structure and see what available stats are useful
- Goal: Test endpoint load and determine reliability

In [1]:
# check game and schedule data
from nba_api.stats.endpoints.scheduleleaguev2 import ScheduleLeagueV2
import pandas as pd


In [37]:
schedule = ScheduleLeagueV2(season="2024-25", league_id="00")
df = schedule.get_data_frames()[0]
df.iloc[100:120]

,leagueId,seasonYear,gameDate,gameId,gameCode,gameStatus,gameStatusText,gameSequence,gameDateEst,gameTimeEst,...,awayOttBroadcasters_broadcasterScope,awayOttBroadcasters_broadcasterMedia,awayOttBroadcasters_broadcasterId,awayOttBroadcasters_broadcasterDisplay,awayOttBroadcasters_broadcasterAbbreviation,awayOttBroadcasters_broadcasterDescription,awayOttBroadcasters_tapeDelayComments,awayOttBroadcasters_broadcasterVideoLink,awayOttBroadcasters_broadcasterTeamId,awayOttBroadcasters_broadcasterRanking
100,00,2024-25,10/26/2024 00:00:00,0022400088,20241026/MIACHA,3,Final,2,2024-10-26T00:00:00Z,1900-01-01T19:00:00Z,...,None,None,NaN,None,None,None,None,None,NaN,NaN
101,00,2024-25,10/26/2024 00:00:00,0022400089,20241026/BOSDET,3,Final,3,2024-10-26T00:00:00Z,1900-01-01T19:00:00Z,...,None,None,NaN,None,None,None,None,None,NaN,NaN
102,00,2024-25,10/26/2024 00:00:00,0022400090,20241026/CLEWAS,3,Final,4,2024-10-26T00:00:00Z,1900-01-01T19:00:00Z,...,None,None,NaN,None,None,None,None,None,NaN,NaN
103,00,2024-25,10/26/2024 00:00:00,0022400091,20241026/OKCCHI,3,Final,5,2024-10-26T00:00:00Z,1900-01-01T20:00:00Z,...,None,None,NaN,None,None,None,None,None,NaN,NaN
104,00,2024-25,10/26/2024 00:00:00,0022400092,20241026/ORLMEM,3,Final,6,2024-10-26T00:00:00Z,1900-01-01T20:00:00Z,...,None,None,NaN,None,None,None,None,None,NaN,NaN
105,00,2024-25,10/26/2024 00:00:00,0022400093,20241026/TORMIN,3,Final,7,2024-10-26T00:00:00Z,1900-01-01T20:00:00Z,...,None,None,NaN,None,None,None,None,None,NaN,NaN
106,00,2024-25,10/26/2024 00:00:00,0022400094,20241026/HOUSAS,3,Final,8,2024-10-26T00:00:00Z,1900-01-01T20:30:00Z,...,None,None,NaN,None,None,None,None,None,NaN,NaN
107,00,2024-25,10/26/2024 00:00:00,0022400095,20241026/DALPHX,3,Final,9,2024-10-26T00:00:00Z,1900-01-01T22:00:00Z,...,away,ott,1001022.0,MavsTV,DTC-DAL,,,,1.610613e+09,0.0
108,00,2024-25,10/26/2024 00:00:00,0022400096,20241026/SACLAL,3,Final,10,2024-10-26T00:00:00Z,1900-01-01T22:30:00Z,...,None,None,NaN,None,None,None,None,None,NaN,NaN
109,00,2024-25,10/27/2024 00:00:00,0022400097,20241027/PHIIND,3,Final/OT,1,2024-10-27T00:00:00Z,1900-01-01T15:30:00Z,...,None,None,NaN,None,None,None,None,None,NaN,NaN


In [21]:
snippet = df.head()[['gameId','gameDate','gameCode', 'gameDateEst', 'gameSequence', 'gameStatusText']]
snippet

,gameId,gameDate,gameCode,gameDateEst,gameSequence,gameStatusText
0,0012400001,10/04/2024 00:00:00,20241004/BOSDEN,2024-10-04T00:00:00Z,1,Final
1,0012400002,10/04/2024 00:00:00,20241004/NZBUTA,2024-10-04T00:00:00Z,2,Final
2,0012400003,10/04/2024 00:00:00,20241004/MINLAL,2024-10-04T00:00:00Z,3,Final
3,0012400004,10/05/2024 00:00:00,20241005/GSWLAC,2024-10-05T00:00:00Z,1,Final
4,0012400005,10/06/2024 00:00:00,20241006/DENBOS,2024-10-06T00:00:00Z,1,Final


In [13]:
snippet.dtypes

gameId          object
gameDate        object
gameCode        object
gameDateEst     object
gameSequence     int64
dtype: object

In [16]:
# ScheduleLeagueV2 displays exhibition games
# will need to cross check teamIds against list of teams
snippet2 = df.head()[["homeTeam_teamId",
            "homeTeam_teamName",
            "homeTeam_teamCity",
            "homeTeam_teamTricode",
            "homeTeam_teamSlug",
            "awayTeam_teamId",
            "awayTeam_teamName",
            "awayTeam_teamCity",
            "awayTeam_teamTricode",
            "awayTeam_teamSlug",]]
snippet2

,homeTeam_teamId,homeTeam_teamName,homeTeam_teamCity,homeTeam_teamTricode,homeTeam_teamSlug,awayTeam_teamId,awayTeam_teamName,awayTeam_teamCity,awayTeam_teamTricode,awayTeam_teamSlug
0,1610612743,Nuggets,Denver,DEN,nuggets,1610612738,Celtics,Boston,BOS,celtics
1,1610612762,Jazz,Utah,UTA,jazz,15020,Breakers,New Zealand,NZB,breakers
2,1610612747,Lakers,Los Angeles,LAL,lakers,1610612750,Timberwolves,Minnesota,MIN,timberwolves
3,1610612746,Clippers,LA,LAC,clippers,1610612744,Warriors,Golden State,GSW,warriors
4,1610612738,Celtics,Boston,BOS,celtics,1610612743,Nuggets,Denver,DEN,nuggets


In [24]:
# check traditional stats availability
from nba_api.stats.endpoints.boxscoretraditionalv3 import BoxScoreTraditionalV3

In [19]:
trad_stats = BoxScoreTraditionalV3(game_id='0012400001')
df_trad = trad_stats.get_data_frames()[0]
df_trad.head()

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints
0,0012400001,1610612738,Boston,Celtics,BOS,celtics,1627759,Jaylen,Brown,J. Brown,...,1,1,2,2,1,1,2,3,8,-8.0
1,0012400001,1610612738,Boston,Celtics,BOS,celtics,1628369,Jayson,Tatum,J. Tatum,...,3,3,6,5,0,0,2,1,12,-8.0
2,0012400001,1610612738,Boston,Celtics,BOS,celtics,1628436,Luke,Kornet,L. Kornet,...,4,7,11,3,1,0,1,1,6,12.0
3,0012400001,1610612738,Boston,Celtics,BOS,celtics,1628401,Derrick,White,D. White,...,0,1,1,0,2,1,3,2,9,-8.0
4,0012400001,1610612738,Boston,Celtics,BOS,celtics,201950,Jrue,Holiday,J. Holiday,...,1,0,1,0,0,0,0,0,4,-7.0


In [22]:
# will need to convert mm:ss format to usable integer data
snippet_trad = df_trad.head()[["personId",
                               "minutes",
            "fieldGoalsMade",
            "fieldGoalsAttempted",
            "fieldGoalsPercentage",
            "threePointersMade",
            "threePointersAttempted",
            "threePointersPercentage",
            "freeThrowsMade",
            "freeThrowsAttempted",
            "freeThrowsPercentage",
            "reboundsOffensive",
            "reboundsDefensive",
            "reboundsTotal",
            "assists",
            "steals",
            "blocks", 
            "turnovers",
            "foulsPersonal",
            "points",
            "plusMinusPoints"]]
snippet_trad

,personId,minutes,fieldGoalsMade,fieldGoalsAttempted,fieldGoalsPercentage,threePointersMade,threePointersAttempted,threePointersPercentage,freeThrowsMade,freeThrowsAttempted,...,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints
0,1627759,19:04,3,9,0.333,1,5,0.200,1,2,...,1,1,2,2,1,1,2,3,8,-8.0
1,1628369,19:04,4,11,0.364,3,7,0.429,1,2,...,3,3,6,5,0,0,2,1,12,-8.0
2,1628436,21:10,3,5,0.600,0,0,0.000,0,0,...,4,7,11,3,1,0,1,1,6,12.0
3,1628401,19:04,3,9,0.333,1,4,0.250,2,2,...,0,1,1,0,2,1,3,2,9,-8.0
4,201950,12:00,2,7,0.286,0,4,0.000,0,0,...,1,0,1,0,0,0,0,0,4,-7.0


In [23]:
df_trad_pb = trad_stats.get_data_frames()[1]
df_trad_pb.head()

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,minutes,fieldGoalsMade,fieldGoalsAttempted,fieldGoalsPercentage,...,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,startersBench
0,0012400001,1610612743,Denver,Nuggets,DEN,nuggets,75:04,16,33,0.485,...,6,13,19,10,8,1,8,5,41,Starters
1,0012400001,1610612743,Denver,Nuggets,DEN,nuggets,75:04,21,53,0.396,...,6,24,30,19,2,1,15,19,62,Bench
2,0012400001,1610612738,Boston,Celtics,BOS,celtics,90:22,15,41,0.366,...,9,12,21,10,4,2,8,7,39,Starters
3,0012400001,1610612738,Boston,Celtics,BOS,celtics,90:22,22,57,0.386,...,8,15,23,19,9,6,9,17,68,Bench


In [25]:
# check advanced stat availability
from nba_api.stats.endpoints.boxscoreadvancedv3 import BoxScoreAdvancedV3

In [29]:
adv_stats = BoxScoreAdvancedV3(game_id='0012400001')
df_adv = adv_stats.get_data_frames()[0]
df_adv.head()

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,turnoverRatio,effectiveFieldGoalPercentage,trueShootingPercentage,usagePercentage,estimatedUsagePercentage,estimatedPace,pace,pacePer40,possessions,PIE
0,0012400001,1610612738,Boston,Celtics,BOS,celtics,1627759,Jaylen,Brown,J. Brown,...,14.3,0.389,0.405,0.203,0.212,124.67,118.32,98.60,48.0,0.012
1,0012400001,1610612738,Boston,Celtics,BOS,celtics,1628369,Jayson,Tatum,J. Tatum,...,10.5,0.500,0.505,0.237,0.247,124.67,118.32,98.60,48.0,0.130
2,0012400001,1610612738,Boston,Celtics,BOS,celtics,1628436,Luke,Kornet,L. Kornet,...,11.1,0.600,0.600,0.107,0.108,110.26,108.85,90.71,47.0,0.217
3,0012400001,1610612738,Boston,Celtics,BOS,celtics,1628401,Derrick,White,D. White,...,23.1,0.389,0.455,0.220,0.230,124.67,118.32,98.60,48.0,0.019
4,0012400001,1610612738,Boston,Celtics,BOS,celtics,201950,Jrue,Holiday,J. Holiday,...,0.0,0.286,0.286,0.200,0.207,125.68,118.00,98.33,30.0,-0.011


In [30]:
snipped_adv = df_adv.head()[["personId", 
            "firstName", 
            "familyName",
             "minutes", 
            "estimatedOffensiveRating", 
            "offensiveRating", 
            "estimatedDefensiveRating", 
            "defensiveRating", 
            "estimatedNetRating", 
            "netRating", 
            "assistPercentage", 
            "assistToTurnover", 
            "assistRatio", 
            "offensiveReboundPercentage", 
            "defensiveReboundPercentage", 
            "reboundPercentage", 
            "turnoverRatio", 
            "effectiveFieldGoalPercentage", 
            "trueShootingPercentage", 
            "usagePercentage", 
            "estimatedUsagePercentage", 
            "estimatedPace", 
            "pace", 
            "pacePer40", 
            "possessions", 
            "PIE"]]
snipped_adv

,personId,firstName,familyName,minutes,estimatedOffensiveRating,offensiveRating,estimatedDefensiveRating,defensiveRating,estimatedNetRating,netRating,...,turnoverRatio,effectiveFieldGoalPercentage,trueShootingPercentage,usagePercentage,estimatedUsagePercentage,estimatedPace,pace,pacePer40,possessions,PIE
0,1627759,Jaylen,Brown,19:04,90.5,91.7,103.2,113.0,-12.7,-21.4,...,14.3,0.389,0.405,0.203,0.212,124.67,118.32,98.60,48.0,0.012
1,1628369,Jayson,Tatum,19:04,90.5,91.7,103.2,113.0,-12.7,-21.4,...,10.5,0.500,0.505,0.237,0.247,124.67,118.32,98.60,48.0,0.130
2,1628436,Luke,Kornet,21:10,111.3,110.6,79.2,81.6,32.1,29.0,...,11.1,0.600,0.600,0.107,0.108,110.26,108.85,90.71,47.0,0.217
3,1628401,Derrick,White,19:04,90.5,91.7,103.2,113.0,-12.7,-21.4,...,23.1,0.389,0.455,0.220,0.230,124.67,118.32,98.60,48.0,0.019
4,201950,Jrue,Holiday,12:00,83.7,83.3,97.1,110.3,-13.4,-27.0,...,0.0,0.286,0.286,0.200,0.207,125.68,118.00,98.33,30.0,-0.011


In [31]:
# check defensive endpoint
from nba_api.stats.endpoints.boxscoredefensivev2 import BoxScoreDefensiveV2

In [46]:
# defensive endpoint not available for preseason games (can check with game id prefixed as '002')
def_stats = BoxScoreDefensiveV2(game_id='0022400088')
df_def = def_stats.get_data_frames()[0]
df_def.head()

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,matchupAssists,matchupTurnovers,steals,blocks,matchupFieldGoalsMade,matchupFieldGoalsAttempted,matchupFieldGoalPercentage,matchupThreePointersMade,matchupThreePointersAttempted,matchupThreePointerPercentage
0,0022400088,1610612748,Miami,Heat,MIA,heat,202710,Jimmy,Butler III,J. Butler III,...,4,1,2,1,5,9,0.556,2,4,0.500
1,0022400088,1610612748,Miami,Heat,MIA,heat,1631107,Nikola,Jović,N. Jović,...,1,2,2,0,3,13,0.231,1,8,0.125
2,0022400088,1610612748,Miami,Heat,MIA,heat,1628389,Bam,Adebayo,B. Adebayo,...,5,3,1,1,3,10,0.300,2,4,0.500
3,0022400088,1610612748,Miami,Heat,MIA,heat,1629639,Tyler,Herro,T. Herro,...,1,2,1,0,9,15,0.600,2,7,0.286
4,0022400088,1610612748,Miami,Heat,MIA,heat,1626179,Terry,Rozier,T. Rozier,...,1,1,0,0,10,17,0.588,3,7,0.429


In [45]:
df_def_2 = def_stats.get_data_frames()[1]
df_def_2

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,minutes
0,0022400088,1610612766,Charlotte,Hornets,CHA,hornets,None
1,0022400088,1610612748,Miami,Heat,MIA,heat,None


In [47]:
# check four factors stats
from nba_api.stats.endpoints.boxscorefourfactorsv3 import BoxScoreFourFactorsV3

In [49]:
ff_stats = BoxScoreFourFactorsV3(game_id='0022400088')
df_ff = ff_stats.get_data_frames()[0]
df_ff.head()

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,jerseyNum,minutes,effectiveFieldGoalPercentage,freeThrowAttemptRate,teamTurnoverPercentage,offensiveReboundPercentage,oppEffectiveFieldGoalPercentage,oppFreeThrowAttemptRate,oppTeamTurnoverPercentage,oppOffensiveReboundPercentage
0,0022400088,1610612748,Miami,Heat,MIA,heat,202710,Jimmy,Butler III,J. Butler III,...,22,36:54,0.523,0.328,0.148,0.105,0.537,0.250,0.178,0.367
1,0022400088,1610612748,Miami,Heat,MIA,heat,1631107,Nikola,Jović,N. Jović,...,5,29:51,0.509,0.263,0.124,0.000,0.456,0.193,0.194,0.250
2,0022400088,1610612748,Miami,Heat,MIA,heat,1628389,Bam,Adebayo,B. Adebayo,...,13,32:37,0.508,0.262,0.119,0.108,0.524,0.206,0.167,0.290
3,0022400088,1610612748,Miami,Heat,MIA,heat,1629639,Tyler,Herro,T. Herro,...,14,32:37,0.508,0.262,0.119,0.027,0.524,0.206,0.167,0.290
4,0022400088,1610612748,Miami,Heat,MIA,heat,1626179,Terry,Rozier,T. Rozier,...,2,31:36,0.508,0.262,0.106,0.000,0.517,0.220,0.172,0.241


In [54]:
# stress testing on endpoints
schedule = ScheduleLeagueV2(season="2024-25", league_id="00")
df_schedule = schedule.get_data_frames()[0]
game_ids = df_schedule.iloc[100:120]['gameId'].tolist()

In [53]:
from nba_api.stats.endpoints.boxscoretraditionalv3 import BoxScoreTraditionalV3
from nba_api.stats.endpoints.boxscoreadvancedv3 import BoxScoreAdvancedV3
from nba_api.stats.endpoints.boxscoredefensivev2 import BoxScoreDefensiveV2
from nba_api.stats.endpoints.boxscorefourfactorsv3 import BoxScoreFourFactorsV3
from nba_api.stats.endpoints.boxscoreplayertrackv3 import BoxScorePlayerTrackV3
from nba_api.stats.endpoints.boxscoreusagev3 import BoxScoreUsageV3
from nba_api.stats.endpoints.boxscorehustlev2 import BoxScoreHustleV2
from nba_api.stats.endpoints.boxscorematchupsv3 import BoxScoreMatchupsV3
from nba_api.stats.endpoints.boxscoremiscv3 import BoxScoreMiscV3

endpoints = [
    BoxScoreTraditionalV3,
    BoxScoreAdvancedV3,
    BoxScoreFourFactorsV3,
    BoxScoreDefensiveV2,
    BoxScoreMiscV3,
    BoxScorePlayerTrackV3,
    BoxScoreHustleV2,
    BoxScoreUsageV3,
    BoxScoreMatchupsV3
]

In [56]:
import time, random

def polite_sleep(base_delay=1.5):
    time.sleep(base_delay + random.uniform(0, 0.5))

In [57]:
results = []

for game_id in game_ids:
    print(f"Testing game: {game_id}")
    for ep in endpoints:
        start = time.time()
        try:
            ep(game_id=game_id)
            status = "success"
        except Exception as e:
            status = f"error: {e}"
        elapsed = round(time.time() - start, 2)
        results.append((game_id, ep.__name__, status, elapsed))
        print(f"  {ep.__name__} | {status} | {elapsed}s")
        polite_sleep()

Testing game: 0022400088
  BoxScoreTraditionalV3 | success | 0.31s
  BoxScoreAdvancedV3 | success | 1.47s
  BoxScoreFourFactorsV3 | success | 0.31s
  BoxScoreDefensiveV2 | success | 1.18s
  BoxScoreMiscV3 | success | 0.81s
  BoxScorePlayerTrackV3 | success | 0.82s
  BoxScoreHustleV2 | success | 1.78s
  BoxScoreUsageV3 | success | 0.96s
  BoxScoreMatchupsV3 | success | 0.35s
Testing game: 0022400089
  BoxScoreTraditionalV3 | success | 0.2s
  BoxScoreAdvancedV3 | success | 0.99s
  BoxScoreFourFactorsV3 | success | 2.0s
  BoxScoreDefensiveV2 | success | 0.59s
  BoxScoreMiscV3 | success | 0.92s
  BoxScorePlayerTrackV3 | success | 0.75s
  BoxScoreHustleV2 | success | 1.29s
  BoxScoreUsageV3 | success | 26.01s
  BoxScoreMatchupsV3 | success | 0.29s
Testing game: 0022400090
  BoxScoreTraditionalV3 | success | 0.44s
  BoxScoreAdvancedV3 | success | 1.02s
  BoxScoreFourFactorsV3 | success | 13.2s
  BoxScoreDefensiveV2 | success | 0.53s
  BoxScoreMiscV3 | success | 2.78s
  BoxScorePlayerTrackV3 